In [1]:
from __future__ import division
%load_ext autoreload
%autoreload 2
from django.conf import settings
import cPickle as pickle
import csv
import os
import sys
import numpy as np
import pandas as pd
sys.path.append('c:\dev\opinion\opinion\python\opinion')
import utils.fileutils as fileutils
import utils.metrics as metrics
import nltk

In [2]:
rootdir = r"C:\dev\opinion\papers\sentiplus"
indir = os.path.join(rootdir, "Hybrid")

In [26]:
# Hot Encode DSOSE values and then Change the previous Base file into the new one
#infile = os.path.join(indir, "DSOSE_ResultsConsolidated_RF.xls")
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")

df = pd.read_excel(infile, "Sheet1", encoding="ISO-8859-1")
dsoseHotEncodes = []
for index, row in df.iterrows():
    dsose = row["DSOSE"]
    if dsose == "p":
        dsoseHotEncodes.append(1)
    elif dsose == "n":
        dsoseHotEncodes.append(-1)
    else:
        dsoseHotEncodes.append(0)
df["DSOSE_HotEncoded"] = (pd.Series(dsoseHotEncodes)).values
df.to_excel(infile, index=False)

In [6]:
def computePerformanceOverallOfLearner(infile, learnerCol, filenames):
    #infile = os.path.join(indir, "ResultsConsolidated_"+algo+".xls")
    df = fileutils.readExcel(infile, "Sheet1", encoding="ISO-8859-1")
    exps = []
    gots = []
    labels = set()
    for index, row in df.iterrows():
        fname = row["File"]
        fname = fname.split("_")[0]
        if fname not in filenames:
                #print fname, " not in filenmaes"
                #return
                continue
        else:
                exp = row["ManualLabel"]
                got = row[learnerCol]
                labels.add(exp)
                exps.append(exp)
                gots.append(got)
    computer = metrics.PerformanceMultiClass(exps, gots, labels = list(labels))
    for label in labels:
            pr = computer.precision(label)
            re = computer.recall(label)
            f1 = 2*pr*re/(pr+re)
            print "Label = %s. Precision = %.3f. Recall = %.3f. F1 = %.3f"%(label, pr, re, f1)
    f1_macro = computer.f1_macro_average()
    pr_macro = computer.precision_macro_average()
    rec_macro = computer.recall_macro_average()
    f1_micro, _, _ = computer.compute_micro_average()
    print "F1 Macro = %.3f. Micro = %.3f"%(f1_macro, f1_micro)
    print "Macro Precision = %.3f. Recall = %.3f"%(pr_macro, rec_macro)
    print "-------------------------------"

def computePerformancOfLearner(infile, learnerCol, filenames):
    #infile = os.path.join(indir, "ResultsConsolidated_"+algo+".xls")
    df = fileutils.readExcel(infile, "Sheet1", encoding="ISO-8859-1")
    exps = dict()
    gots = dict()
    labels = dict()
    for index, row in df.iterrows():
        fname = row["File"]
        fname = fname.split("_")[0]
        if fname not in filenames:
                #print fname, " not in filenmaes"
                continue
        else:
                if fname not in exps:
                    exps[fname] = []
                    gots[fname] = []
                    labels[fname] = set()
        exp = row["ManualLabel"]
        got = row[learnerCol]
        labels[fname].add(exp)
        exps[fname].append(exp)
        gots[fname].append(got)
    for fname in filenames:
        computer = metrics.PerformanceMultiClass(exps[fname], gots[fname], labels = list(labels[fname]))
        for label in labels[fname]:
                pr = computer.precision(label)
                re = computer.recall(label)
                f1 = 2*pr*re/(pr+re)
                print "File %s. Label = %s. Precision = %.2f. Recall = %.2f. F1 = %.2f"%(fname, label, pr, re, f1)
        f1_macro = computer.f1_macro_average()
        f1_micro, _, _ = computer.compute_micro_average()
        print "File = %s. F1 Macro = %.2f. Micro = %.2f"%(fname, f1_macro, f1_micro)
        print "-------------------------------"


In [5]:
filenames = ["DatasetLinJIRA", "BenchmarkUddinSO", "DatasetLinAppReviews", 
                     "DatasetLinSO", "DatasetSenti4SDSO", "OrtuJIRA"]

In [25]:
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")
learnerCol = "DsoLabelFullTextW2V"
print learnerCol, ": Overall Performance"
print "-"*80
computePerformanceOverallOfLearner(infile, learnerCol, filenames)    
print learnerCol, ": By File Performance"
print "-"*80
computePerformancOfLearner(infile, learnerCol, filenames)

DsoLabelFullTextW2V : Overall Performance
--------------------------------------------------------------------------------
Label = p. Precision = 0.442. Recall = 0.540. F1 = 0.486
Label = o. Precision = 0.665. Recall = 0.627. F1 = 0.645
Label = n. Precision = 0.528. Recall = 0.469. F1 = 0.497
F1 Macro = 0.545. Micro = 0.572
Macro Precision = 0.545. Recall = 0.545
-------------------------------
DsoLabelFullTextW2V : By File Performance
--------------------------------------------------------------------------------
File DatasetLinJIRA. Label = p. Precision = 0.58. Recall = 0.95. F1 = 0.72
File DatasetLinJIRA. Label = n. Precision = 0.98. Recall = 0.83. F1 = 0.90
File = DatasetLinJIRA. F1 Macro = 0.83. Micro = 0.85
-------------------------------
File BenchmarkUddinSO. Label = p. Precision = 0.34. Recall = 0.50. F1 = 0.40
File BenchmarkUddinSO. Label = o. Precision = 0.68. Recall = 0.63. F1 = 0.65
File BenchmarkUddinSO. Label = n. Precision = 0.37. Recall = 0.25. F1 = 0.30
File = Benchm

In [16]:
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")
learnerCol = "DsoLabelFullText"
print learnerCol, ": Overall Performance"
print "-"*80
computePerformanceOverallOfLearner(infile, learnerCol, filenames)    
print learnerCol, ": By File Performance"
print "-"*80
computePerformancOfLearner(infile, learnerCol, filenames)

DsoLabelFullText : Overall Performance
--------------------------------------------------------------------------------
Label = p. Precision = 0.467. Recall = 0.524. F1 = 0.494
Label = o. Precision = 0.668. Recall = 0.684. F1 = 0.676
Label = n. Precision = 0.577. Recall = 0.460. F1 = 0.512
F1 Macro = 0.563. Micro = 0.597
Macro Precision = 0.571. Recall = 0.556
-------------------------------
DsoLabelFullText : By File Performance
--------------------------------------------------------------------------------
File DatasetLinJIRA. Label = p. Precision = 0.60. Recall = 0.94. F1 = 0.73
File DatasetLinJIRA. Label = n. Precision = 0.98. Recall = 0.84. F1 = 0.91
File = DatasetLinJIRA. F1 Macro = 0.84. Micro = 0.86
-------------------------------
File BenchmarkUddinSO. Label = p. Precision = 0.36. Recall = 0.45. F1 = 0.40
File BenchmarkUddinSO. Label = o. Precision = 0.67. Recall = 0.70. F1 = 0.69
File BenchmarkUddinSO. Label = n. Precision = 0.43. Recall = 0.23. F1 = 0.30
File = BenchmarkUdd

In [13]:
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")
learnerCol = "Senti4SD"
print learnerCol, ": Overall Performance"
print "-"*80
computePerformanceOverallOfLearner(infile, learnerCol, filenames)    
print learnerCol, ": By File Performance"
print "-"*80
computePerformancOfLearner(infile, learnerCol, filenames)

Senti4SD : Overall Performance
--------------------------------------------------------------------------------
Label = p. Precision = 0.773. Recall = 0.692. F1 = 0.731
Label = o. Precision = 0.783. Recall = 0.865. F1 = 0.822
Label = n. Precision = 0.748. Recall = 0.640. F1 = 0.690
F1 Macro = 0.750. Micro = 0.774
Macro Precision = 0.768. Recall = 0.732
-------------------------------
Senti4SD : By File Performance
--------------------------------------------------------------------------------
File DatasetLinJIRA. Label = p. Precision = 0.91. Recall = 0.94. F1 = 0.93
File DatasetLinJIRA. Label = n. Precision = 0.97. Recall = 0.96. F1 = 0.97
File = DatasetLinJIRA. F1 Macro = 0.95. Micro = 0.95
-------------------------------
File BenchmarkUddinSO. Label = p. Precision = 0.48. Recall = 0.29. F1 = 0.36
File BenchmarkUddinSO. Label = o. Precision = 0.66. Recall = 0.85. F1 = 0.75
File BenchmarkUddinSO. Label = n. Precision = 0.46. Recall = 0.27. F1 = 0.34
File = BenchmarkUddinSO. F1 Macro =

C:\anaconda3\envs\opinion2\lib\site-packages\ipykernel_launcher.py:61: RuntimeWarning: invalid value encountered in double_scalars


In [14]:
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")
learnerCol = "SentiCR"
print learnerCol, ": Overall Performance"
print "-"*80
computePerformanceOverallOfLearner(infile, learnerCol, filenames)    
print learnerCol, ": By File Performance"
print "-"*80
computePerformancOfLearner(infile, learnerCol, filenames)

SentiCR : Overall Performance
--------------------------------------------------------------------------------
Label = p. Precision = 0.803. Recall = 0.667. F1 = 0.728
Label = o. Precision = 0.767. Recall = 0.903. F1 = 0.830
Label = n. Precision = 0.804. Recall = 0.599. F1 = 0.687
F1 Macro = 0.756. Micro = 0.780
Macro Precision = 0.791. Recall = 0.723
-------------------------------
SentiCR : By File Performance
--------------------------------------------------------------------------------
File DatasetLinJIRA. Label = p. Precision = 0.97. Recall = 0.84. F1 = 0.90
File DatasetLinJIRA. Label = n. Precision = 0.93. Recall = 0.99. F1 = 0.96
File = DatasetLinJIRA. F1 Macro = 0.93. Micro = 0.94
-------------------------------
File BenchmarkUddinSO. Label = p. Precision = 0.47. Recall = 0.15. F1 = 0.23
File BenchmarkUddinSO. Label = o. Precision = 0.62. Recall = 0.91. F1 = 0.74
File BenchmarkUddinSO. Label = n. Precision = 0.42. Recall = 0.17. F1 = 0.24
File = BenchmarkUddinSO. F1 Macro = 0

In [15]:
infile = os.path.join(indir, "ConsolidatedWithBestTrainedClfsFromThreeSETools.xls")
learnerCol = "SentistrengthSE"
print learnerCol, ": Overall Performance"
print "-"*80
computePerformanceOverallOfLearner(infile, learnerCol, filenames)    
print learnerCol, ": By File Performance"
print "-"*80
computePerformancOfLearner(infile, learnerCol, filenames)

SentistrengthSE : Overall Performance
--------------------------------------------------------------------------------
Label = p. Precision = 0.752. Recall = 0.690. F1 = 0.720
Label = o. Precision = 0.750. Recall = 0.847. F1 = 0.795
Label = n. Precision = 0.736. Recall = 0.566. F1 = 0.640
F1 Macro = 0.723. Micro = 0.748
Macro Precision = 0.746. Recall = 0.701
-------------------------------
SentistrengthSE : By File Performance
--------------------------------------------------------------------------------
File DatasetLinJIRA. Label = p. Precision = 0.95. Recall = 0.99. F1 = 0.97
File DatasetLinJIRA. Label = n. Precision = 1.00. Recall = 0.97. F1 = 0.98
File = DatasetLinJIRA. F1 Macro = 0.98. Micro = 0.98
-------------------------------
File BenchmarkUddinSO. Label = p. Precision = 0.49. Recall = 0.22. F1 = 0.31
File BenchmarkUddinSO. Label = o. Precision = 0.63. Recall = 0.89. F1 = 0.74
File BenchmarkUddinSO. Label = n. Precision = 0.43. Recall = 0.15. F1 = 0.23
File = BenchmarkUddin